In [190]:
import pandas as pd
import numpy as np
import datetime as dt
pd.options.mode.chained_assignment = None
import sklearn
from sklearn import linear_model
from scipy.optimize import curve_fit
import math


In [28]:
df_accidents = pd.read_csv("Accidents0514.csv",low_memory=False)
df_casualities = pd.read_csv("Casualties0514.csv",low_memory=False)
df_vehicles = pd.read_csv("Vehicles0514.csv",low_memory=False)

In [ ]:
df_accidents.info()

In [ ]:
#What fraction of accidents occur in urban areas?
df_accidents["Urban_or_Rural_Area"].unique()



In [ ]:
#as we could see there are three unique values for Urban_or_Rural_Area and urban = 1 from data information sheet
#now group by "Urban_or_Rural_Area" to get counts for each area
area_count =df_accidents.groupby(["Urban_or_Rural_Area"])["Accident_Index"].count()

In [ ]:
area_count

In [ ]:
#create a list 
list_area_count= list(area_count)

In [ ]:
#now fraction of accidents in urban area were uraban is first element in list
frac_accidents_urban = list_area_count[0]/sum(list_area_count)

In [ ]:
print (frac_accidents_urban) 

In [ ]:
#drop all the null values in Time column
data = df_accidents.dropna(subset=['Time'])

In [ ]:
def hour(x):
    return x[0]
def minute(x):
    return x[1]
def combine(x):
    return x[0]+x[1]

In [ ]:
data["Time_Hour"] = data["Time"].str.split(":").apply(hour)
data["Time_Min"] = data["Time"].str.split(":").apply(minute)
data["Time_Combined"] = data["Time"].str.split(":").apply(combine)

In [ ]:
group_hour = data.groupby(["Time_Hour"])

In [ ]:
#list of total number of accidents each hour from 00 to 23
total_per_hour = list(group_hour["Time_Combined"].count())

In [ ]:
group_time = data.groupby(["Time_Combined"])["Time_Hour"]

In [ ]:
dict_grp_time = dict(list(group_time))
len(dict_grp_time)

In [ ]:
#calculate the frequency for each distinct timeunit
max_frequency = 0
max_time = 0
for key in dict_grp_time:
    two_digit=[]
    frequency = 0
    i = 0
    two_digit.append(key[i])
    two_digit.append(key[i+1])
    num_accidents_time = len(dict_grp_time[key])
    num_accidents_hour = total_per_hour[int("".join(two_digit))]  
    frequency = (num_accidents_time/num_accidents_hour)
    if max_frequency <  frequency:
        max_frequency = frequency
        max_time = key
    
    

In [ ]:
#submit the corresponding highest frequency of fatal accidents to all accidents in that hour
print (max_frequency)

In [ ]:
#the corresponding most dangerous time to drive is
print (max_time[0:2]+":"+ max_time[2:4])

In [ ]:
#get the year attribute from the date 
df_accidents["Year"] = pd.to_datetime(df_accidents["Date"]).dt.year

In [ ]:
group_year = df_accidents.groupby(["Year"])["Accident_Index"]

In [ ]:
dict_year = dict(list(group_year)) 

In [ ]:
year = []
accident_number = []
for key in dict_year:
    year.append(key)
    accident_number.append(len(dict_year[key]))


In [ ]:
year

In [ ]:
accident_number

In [ ]:
rgr = linear_model.LinearRegression()

In [ ]:
model = rgr.fit(np.array(year).reshape(-1,1),np.array(accident_number))

In [ ]:
#thus the slope is given by
print (model.coef_[0])

In [ ]:
year


In [ ]:
# create a list of count of accidents by speed limit
accidents = list(df_accidents.groupby(["Speed_limit"])["Accident_Index"].count())

In [ ]:
# create a list of sum of casulaities by speed limit
casualities = list(df_accidents.groupby(["Speed_limit"])["Number_of_Casualties"].sum())

In [ ]:
casualities

In [ ]:
accidents

In [ ]:
#now get the ratio of casualities_by_accidentsper speed_limit
casualities_by_accidents = np.array(casualities)/np.array(accidents )

In [ ]:
casualities_by_accidents

In [ ]:
#get the unique values for speed limit
speed_limit = np.array(sorted(list(df_accidents["Speed_limit"].unique())))

In [ ]:
speed_limit

In [ ]:
#create a dataframe with two colums for correlation computaion
df = pd.DataFrame({'x':speed_limit , 'y':casualities_by_accidents})

In [ ]:
#now get the pearson correlation coefficient
print (df['x'].corr(df['y']))

In [76]:
#merge the accidents and vehicles data for skid, jacknifed and overturned
df_merged_av = pd.merge(df_accidents, df_vehicles, on=['Accident_Index'])

In [81]:
#extract the data with rain or snow and fine with no winds seperately
df_snow_rain = df_merged_av[(df_merged_av.Weather_Conditions == 2)|(df_merged_av.Weather_Conditions == 3)|(df_merged_av.Weather_Conditions == 5)|(df_merged_av.Weather_Conditions == 6)]
df_fine_nhw = df_merged_av[df_merged_av.Weather_Conditions == 1]

In [91]:
#group by skidding_overturning
dict_sr_so = dict(list(df_snow_rain.groupby(["Skidding_and_Overturning"])["Weather_Conditions"]))
dict_fnhw_so = dict(list(df_fine_nhw.groupby(["Skidding_and_Overturning"])["Weather_Conditions"]))

In [93]:
r_snow_rain= (len(dict_sr_so[1])+len(dict_sr_so[2])+len(dict_sr_so[3])+len(dict_sr_so[4])+len(dict_sr_so[5]))/len(dict_sr_so[0])

In [96]:
r_fnhw = (len(dict_fnhw_so[1])+len(dict_fnhw_so[2])+len(dict_fnhw_so[3])+len(dict_fnhw_so[4])+len(dict_fnhw_so[5]))/len(dict_fnhw_so[0])

In [99]:
ratio_sr_fnhw = r_snow_rain/r_fnhw 

In [100]:
print (ratio_sr_fnhw)

1.9867686197709724


In [98]:
df_fine_nhw.groupby(["Skidding_and_Overturning"])["Weather_Conditions"].count()

Skidding_and_Overturning
-1         91
 0    2124043
 1     200875
 2      43691
 3        897
 4        567
 5      35770
Name: Weather_Conditions, dtype: int64

In [64]:
#merge the vehicles and casualities data for sex fatality detection
df_merged_vc = pd.merge(df_vehicles,df_casualities, on=['Accident_Index', 'Vehicle_Reference'])


In [66]:
#extarct only rows with car vehicle type = 9
df_car = df_merged_vc[df_merged_vc.Vehicle_Type == 9]

In [67]:
#extract only rows with Cmale and female seperately
df_male = df_car[df_car.Sex_of_Driver == 1]
df_female = df_car[df_car.Sex_of_Driver == 2]

In [68]:
#extract only rows with Casualty_Severity = Fatal = 1
dict_male = dict(list(df_male.groupby(["Casualty_Severity"])))
dict_female = dict(list(df_female.groupby(["Casualty_Severity"])))

In [69]:
df_female.groupby(["Casualty_Severity"]).count()

,Accident_Index,Vehicle_Reference,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type
Casualty_Severity,,,,,,,,,,,,,,,,,,,,,
1,3291,3291,3291,3291,3291,3291,3291,3291,3291,3291,...,3291,3291,3291,3291,3291,3291,3291,3291,3291,3291
2,44290,44290,44290,44290,44290,44290,44290,44290,44290,44290,...,44290,44290,44290,44290,44290,44290,44290,44290,44290,44290
3,591724,591724,591724,591724,591724,591724,591724,591724,591724,591724,...,591724,591724,591724,591724,591724,591724,591724,591724,591724,591724


In [70]:
#male 1 and female 2
male_rate = len(dict_male[1])/(len(dict_male[1])+len(dict_male[2])+len(dict_male[3]))

In [71]:
female_rate = len(dict_female[1])/(len(dict_female[1])+len(dict_female[2])+len(dict_female[3]))

In [72]:
female_rate

0.005147777664807877

In [73]:
ratio_maleFemale = male_rate/female_rate

In [74]:
print (ratio_maleFemale)

2.3090925540687834


In [103]:
#selecting only drivers >= 17
df_age = df_merged_av[(df_merged_av.Age_of_Driver >= 17 )]

In [109]:
dict_age = dict(list(df_age.groupby(["Age_of_Driver"])["Accident_Index"]))

In [111]:
#get array of age and acciden_count variables
age = []
accidents = []
for i in dict_age:
    age.append(i)
    accidents.append(len(dict_age[i]))

In [149]:
X = np.array(age)
Y = np.array(accidents)
param, cov = curve_fit(lambda x,a,b: a*np.exp(b*x),  X,  Y,  p0=(1, 0.1))

In [151]:
# to get the rate of in increase now rate of decay will be -1*rate of increase
print (param[1])


-0.0286469846105


In [155]:
#drop all the null values in logitude and latitude columns
data_area = df_accidents.dropna(subset=['Longitude','Latitude'])

In [182]:
#group by Local_Authority_(District) and calculate standard deviation for longitude and latitude
df_long_std = data_area.groupby(["Local_Authority_(District)"]).Longitude.agg(lambda x: np.std(x))
df_lat_std = data_area.groupby(["Local_Authority_(District)"]).Latitude.agg(lambda x: np.std(x))


In [188]:
long_std = list(df_long_std)
lat_std = list(df_lat_std)


In [192]:
#function to get list of all areas
def get_area(x,y):
    area = []
    for i in range(len(x)):
        area.append(math.pi*x[i]*y[i])
    return area
    

In [193]:
area = get_area(long_std,lat_std)

In [200]:
max_area = max(area)

In [201]:
print (max_area)

1.09117187667
